# Help vision impaired people to see the world

Combining leading edge technologies for helping people – Connect the dots:


1. **Classifying** different **pictures** (tickets, floorplans and documents)
2. **Recognizing text** in the images
3. Reading out loud through **text-to-speech** (English)



##<font color=red>Advice:</font>
<font color=red>To train the network fast, from the Runtime environment Menu, select the GPU Hardware Acceleration.

In Spanish 'Entorno de ejecucion -> Cambiar tipo de entorno de ejecución -> Acelerador por Hardware: GPU'.</font>

## Reading out loud through text to speech

**Tools**:

Text-to-Speech librariesto explore:
* Google Text-To-Speech
* Amazon Polly
* gTTS2.2.2
* pyttsx3 2.90

---
**Goals**

To turn recognized and summarized texts into natural-sounding speech.

The text read should be in English.

---
**Deliverables**

Github Code (master) and Pages (report) Executable:
*   Input: text or caption (.txt)
*   Output: audio (.mp3)

---

**STEPS:**

1. Text-to-Speech



### 1. Text-to-speech
Connect to Drive where the data is stored.

Import all necessary packages.

Define constants like paths.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import necessary packages
! pip install gtts
! pip install IPython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.1 MB/s eta 0:00:00
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
flask 1.1.4 requires click<8.0,>=5.1, but you have cl

In [ ]:
from gtts import gTTS 
from IPython.display import Audio 
import json
from glob import glob

In [ ]:
# specify path to the dataset
ROOT = "/content/drive/MyDrive/ColabNotebooks/"
DOCS_DATASET_PATH = "Database"
NEW_DATA_PATH = 'DatabaseOrdered'
OCR_DATA_PATH = 'boxesAndKey'

ocr_extract = ['boxes', 'key']
boxColPos = ['0', '1', '2', '3', '4', '5', '6', '7']
worldCol = [*(f'word{i:02}' for i in range(1, 10))]

# specify the paths to our training and validation set 
TRAIN = "train"
TEST = "test"

classes = ['Tickets']
doc_type = {'Tickets': 'ticket'}

In [ ]:
img_fns = glob(ROOT + NEW_DATA_PATH + '/' + TRAIN + '/Tickets/*') + glob(ROOT + NEW_DATA_PATH + '/' + TEST + '/Tickets/*')
print(len(img_fns))
print(img_fns[0:2])
print(img_fns[-2:])

626
['/content/drive/MyDrive/ColabNotebooks/DatabaseOrdered/train/Tickets/000.jpg', '/content/drive/MyDrive/ColabNotebooks/DatabaseOrdered/train/Tickets/001.jpg']
['/content/drive/MyDrive/ColabNotebooks/DatabaseOrdered/test/Tickets/624.jpg', '/content/drive/MyDrive/ColabNotebooks/DatabaseOrdered/test/Tickets/625.jpg']


In [ ]:
def get_actual_key_values(image_id):
  image_id = img_fns[image_id].split('/')[-1].split('.')[0]
  keyFile = ROOT+OCR_DATA_PATH+'/'+ocr_extract[1]+'/'+image_id+'.json'
  f = open(keyFile)
  jskey = json.load(f)
  f.close()
  return jskey

In [ ]:
def get_rellevant_info(image_id):
  keyFile = ROOT+'summary'+str(image_id)+'.json'
  f = open(keyFile)
  jskey = json.load(f)
  f.close()
  return jskey

In [ ]:
import datetime
from dateutil import parser
import re

In [ ]:
MONTHS = ["january", "february", "march", "april", "may", "june","july", "august", "september","october", "november", "december"]
DAY_EXTENTIONS = ["st", "nd", "rd", "th"]

In [ ]:
def get_date(text):
    text = text.lower()
    dt =parser.parse(text, dayfirst=True)

    month = MONTHS[dt.month-1]
    day = ordinal(dt.day)
    return str(month + " " + day + " " + str(dt.year))


def ordinal(n):
    suffix = {1:'st', 2:'nd', 3:'rd', 11:'th', 12:'th', 13:'th' }
    return str(n)+(suffix.get(n%100) or suffix.get(n%10,'th'))

def get_money(text):
  qt = str()
  m = re.split(r'\.|,', text)

  qt = str(m[0] + ' euros')
  if len(m)>1 and int(m[1])>0:
    qt += str(' and '+m[1] + ' cents')

  return qt

In [ ]:
d = get_date('19/01/2016')
print(d)
tts = gTTS(d) 
tts.save('my_sentence.wav') #You can change the name of the file
sound_file = 'my_sentence.wav'
Audio(sound_file, autoplay=True) 

january 19th 2016


This is a receipt from Book tk restaurant date at July, 19th 2019. Total amount is tree hundred thirty five euros comming from thirty nineth on single est.

In [ ]:
data = get_rellevant_info(523)
data

{'company': ['UNIHAKKA INTERNATIONAL SDN BHD', 94.5],
 'date': ['10 JUN 2018', 95.5],
 'address': ['12, Jalan Tampoi 7/4,Kawasan Perindustrian Tampoi,81200 Johor Bahru, Johor',
  91.75],
 'total': ['8.48', 92.66666666666667]}

In [ ]:
text = 'This is a receipt'
if data['company'][0] != '':
  text += ' from ' + data['company'][0]

if data['date'][0] != '':
  text += ', date at ' + get_date(data['date'][0]) + '.'

if data['total'][0] != '':
  text += ' Total amount is ' + get_money(data['total'][0]) + '.'

if data['address'][0] != '':
  text += ' Comming from '+ data['address'][0]

print(text)

This is a receipt from UNIHAKKA INTERNATIONAL SDN BHD, date at june 10th 2018. Total amount is 8 euros and 48 cents. Comming from 12, Jalan Tampoi 7/4,Kawasan Perindustrian Tampoi,81200 Johor Bahru, Johor


In [ ]:
tts = gTTS(text) 
tts.save('my_sentence.wav') #You can change the name of the file
sound_file = 'my_sentence.wav'
Audio(sound_file, autoplay=True) 